In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import torch.nn as nn
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from pathlib import Path
from PIL import Image
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [ ]:
print(os.listdir('.'))

['.config', 'drive', 'sample_data']


In [ ]:
%cd  '/content/drive/MyDrive/CV8160 S24/HW1'

/content/drive/MyDrive/CV8160 S24/HW1


In [ ]:
root_dir = '/content/drive/MyDrive/CV8160 S24/HW1'

In [ ]:
!ls

classification	CV8160_HW1.ipynb  localize


In [ ]:
class localize(Dataset):
    def __init__(self, root_dir):

        self.root = Path(root_dir)
        self.df = pd.DataFrame(columns = ['img_name','vclass','x1','y1','x2','y2'])

        for folder in self.root.iterdir()
          img_name = []
          if fldr.is_dir():
            img_name = img_name + list(fldr.glob('*.jpg'))

SyntaxError: expected ':' (<ipython-input-32-a61c658c387a>, line 7)

In [ ]:
class localize(Dataset):
    def __init__(self, root_dir):

        self.root = Path(root_dir)

        #read train.csv
        self.df = pd.read_csv('localize/train.csv')

        #get list of image names in classd folder
        self.classd = root_dir + '/localize/classd'
        self.filename = []
        self.filename = os.listdir(self.classd)
        # self.filename.str.strip('.jpg')
        # self.filename.str.lstrip('0')

        self.df_filename = pd.DataFrame(self.filename)
        #remove leading zeros and .jpg so 'img_name' and 'filename' match
        self.df_filename[0] = self.df_filename[0].str.strip('.jpg')
        self.df_filename[0] = self.df_filename[0].str.lstrip('0')
        self.df_filename.rename(columns={0:'img_name'},inplace=True)

        self.df['img_name'] = self.df['img_name'].astype(str)
        self.df_filename['img_name'] = self.df_filename['img_name'].astype(str)
        self.df = pd.merge(self.df_filename, self.df, on='img_name', how='left')
        # self.df = self.df.rename(columns = ['filename','img_name','vclass','x1','y1','x2','y2'])


        #turn df into lists
        self.img_name = list(self.df['img_name'].values)
        self.vclass   = list(self.df['vclass'].values)
        self.x1       = list(self.df['x1'].values)
        self.y1       = list(self.df['y1'].values)
        self.x2       = list(self.df['x2'].values)
        self.y2       = list(self.df['y2'].values)

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, idx):
        image = self.img_name[idx]
        img_data = io.imread(image)

        label = self.vclass[idx]
        coord_1 = concat([x1[idx],',' ,y1[idx]])
        coord_2 = concat([x2[idx],',' ,y2[idx]])
        sample = {'image':img_data, 'vehicle class':vclass, 'coord_1':coord_1, 'coord_2':coord_2}


In [ ]:
lc = localize(root_dir)
lc.df

,img_name,vclass,x1,y1,x2,y2
0,53505,car,596.0,172.0,652.0,199.0
1,46390,car,318.0,111.0,336.0,124.0
2,39759,articulated_truck,3.0,46.0,104.0,82.0
3,37657,car,301.0,112.0,353.0,129.0
4,3459,single_unit_truck,141.0,15.0,191.0,33.0
...,...,...,...,...,...,...
1004,30806,car,188.0,75.0,295.0,163.0
1005,32549,bus,256.0,158.0,325.0,243.0
1006,2855,car,126.0,257.0,297.0,388.0
1007,28313,car,96.0,22.0,114.0,30.0


In [ ]:
# prompt: flip image and convert to pytorch

def flip_and_convert_image(image):
  """
  Flips the image horizontally and converts it to a PyTorch tensor.

  Args:
    image: A PIL Image object.

  Returns:
    A PyTorch tensor representing the flipped image.
  """
  # Flip the image horizontally
  flipped_image = image.transpose(Image.FLIP_LEFT_RIGHT)

  # Convert the image to a PyTorch tensor
  tensor = transforms.ToTensor()(flipped_image)

  return tensor


In [ ]:
class rescale(object):
  def __init__(self, resolution):
    assert isinstance(resolution, int)
    self.resolution = resolution

  def __call__(self, sample):
      image, labels = sample['image'], sample['label']
      img = np.flip(image, self.resolution)

      return {'image': img, 'label': labels}

class ToTensor(object):
  def __call__(self,sample):
    image, labels = sample['image'], sample['label']
    image = image.transpose(2,0,1)
    image = torch.from_numpy(image)

    return {'image': image, 'label': labels}

In [ ]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, labels = sample['image'], sample['label']

        h, w = image.shape[:2]

        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))
        return sample
        # return {'image': img, 'label': labels}

In [ ]:
class flip_image(object):
  def __init__(self, fliptype):
    assert isinstance(fliptype, int)
    self.fliptype = fliptype

  def __call__(self, sample):
      image, labels = sample['image'], sample['label']
      img = np.flip(image, self.fliptype)

      return {'image': img, 'label': labels}

class ToTensor(object):
  def __call__(self,sample):
    image, labels = sample['image'], sample['label']
    image = image.transpose(2,0,1)
    image = torch.from_numpy(image)

    return {'image': image, 'label': labels}


In [ ]:
fig = plt.figure(figsize=(20,20))
ncols = 4
for i, sample in enumerate(sample):
  # print(i, sample['image'].shape, sample['label'], sample['x1'], sample['y1'], sample['x2'], sample['y2'])
  img = sample['image']
  ax = plt.subplot(1, ncols, i + 1)
  plt.tight_layout()
  ax.set_title('Sample #{} - {} , x1 = {}, y1 = {}, x1 = {}, x2={}'.format(i, sample['label'],sample['x1'], sample['y1'], sample['x2'], sample['y2']))
  ax.axis('off')
  plt.imshow(sample['image'])

  if i == ncols-1:
      plt.show()
      break

TypeError: string indices must be integers

<Figure size 2000x2000 with 0 Axes>

In [ ]:
type(lc.df['img_name'])

pandas.core.series.Series

In [ ]:
lc.df_filename[0]

NameError: name 'lc' is not defined

In [ ]:
lc.df

,0,img_name,vclass,x1,y1,x2,y2
0,00053505.jpg,15647.0,car,382.0,205.0,474.0,264.0
1,00046390.jpg,95025.0,motorized_vehicle,1.0,105.0,59.0,152.0
2,00039759.jpg,2855.0,car,126.0,257.0,297.0,388.0
3,00037657.jpg,56051.0,car,27.0,96.0,37.0,110.0
4,00003459.jpg,30806.0,car,188.0,75.0,295.0,163.0
...,...,...,...,...,...,...,...
1000,00030806.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1001,00032549.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1002,00002855.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1003,00028313.jpg,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
lc.filename

In [ ]:
lc.df

NameError: name 'lc' is not defined

In [ ]:
df = pd.read_csv('localize/train.csv')
filename = os.listdir(classd)
df_filename = pd.DataFrame(filename)
df = pd.concat([df_filename, df], axis=1)
df

,0,img_name,vclass,x1,y1,x2,y2
0,00000369.jpg,15647.0,car,382.0,205.0,474.0,264.0
1,00053505.jpg,95025.0,motorized_vehicle,1.0,105.0,59.0,152.0
2,00046390.jpg,2855.0,car,126.0,257.0,297.0,388.0
3,00039759.jpg,56051.0,car,27.0,96.0,37.0,110.0
4,00037657.jpg,30806.0,car,188.0,75.0,295.0,163.0
...,...,...,...,...,...,...,...
1000,00030806.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1001,00032549.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1002,00002855.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1003,00028313.jpg,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = pd.read_csv('localize/train.csv')
df.head()

img_name = list(df['img_name'].values)
print(img_name)

[15647, 95025, 2855, 56051, 30806, 37657, 10348, 17605, 29251, 32549, 28313, 100833, 53505, 31577, 3459, 38847, 46390, 39759, 6849, 3204, 79494, 25690, 61414, 110584, 71754, 98276, 41731, 61846, 16551, 69228, 98690, 68843, 96320, 43279, 13505, 1939, 3980, 51450, 64196, 31185, 110327, 13483, 51542, 103656, 110153, 107885, 70954, 61262, 15603, 45099, 49719, 65143, 16721, 78288, 89602, 62445, 6018, 88256, 52308, 18038, 97719, 89304, 22266, 43649, 106903, 13118, 39406, 99224, 91280, 88500, 39763, 92501, 34876, 9343, 43888, 106893, 41784, 109668, 92709, 96883, 79629, 35282, 50472, 16487, 16535, 4638, 58254, 64197, 9525, 8636, 100720, 735, 88099, 94246, 74874, 71254, 87359, 63278, 71744, 84982, 77480, 13847, 17980, 22377, 16072, 50033, 94273, 67073, 89407, 68843, 74121, 82016, 71428, 35210, 92573, 100144, 74089, 30970, 1739, 85927, 106581, 40552, 84528, 84992, 69354, 36438, 20696, 90359, 29000, 71448, 44819, 81892, 79172, 102005, 29616, 66840, 69768, 20449, 84521, 97395, 52471, 20759, 65906,

In [ ]:
classd = root_dir + '/localize/classd'
img_name = []
img_name = os.listdir(classd)



In [ ]:
print(img_name)

['00000369.jpg', '00053505.jpg', '00046390.jpg', '00039759.jpg', '00037657.jpg', '00003459.jpg', '00010348.jpg', '00056051.jpg', '00029251.jpg', '00038847.jpg', '00095025.jpg', '00031577.jpg', '00017605.jpg', '00015647.jpg', '00061846.jpg', '00098276.jpg', '00003204.jpg', '00013505.jpg', '00061414.jpg', '00110153.jpg', '00025690.jpg', '00110584.jpg', '00069228.jpg', '00071754.jpg', '00006849.jpg', '00107885.jpg', '00003980.jpg', '00031185.jpg', '00051450.jpg', '00110327.jpg', '00041731.jpg', '00001939.jpg', '00096320.jpg', '00068843.jpg', '00103656.jpg', '00079494.jpg', '00043279.jpg', '00098690.jpg', '00064196.jpg', '00051542.jpg', '00013483.jpg', '00016551.jpg', '00088500.jpg', '00049719.jpg', '00016721.jpg', '00039763.jpg', '00039406.jpg', '00092501.jpg', '00062445.jpg', '00034876.jpg', '00013118.jpg', '00099224.jpg', '00078288.jpg', '00089602.jpg', '00065143.jpg', '00106903.jpg', '00091280.jpg', '00045099.jpg', '00061262.jpg', '00006018.jpg', '00015603.jpg', '00088256.jpg', '000709